In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
import math
from random import shuffle
import random

In [25]:
data = pd.read_csv("./data/kor_201901_202106_additional_classcode_cleaned_outlier_removed_data_over10.csv")

In [26]:
data = data.iloc[:,:12] # remove unnecessary columns

In [27]:
data.shape

(556164, 12)

In [28]:
import pickle
with open("../../save_variables/table_contentid.pickle", "rb") as fp:
    table_id = pickle.load(fp)

In [29]:
t_index = data[data.id.isin(table_id)].index

In [30]:
t_index

Int64Index([ 17532,  17596,  18315,  18512,  18940,  19294,  19399,  19440,
             19459,  19475,
            ...
            554736, 554802, 554845, 555069, 555138, 555177, 555421, 555516,
            555733, 556035],
           dtype='int64', length=3923)

In [ ]:
for t in t_index:
    temp_body = ' '.join(sent_tokenize(data.body[t])[:-1])
    data.body[t] = temp_body

In [32]:
data.shape

(556164, 12)

In [33]:
short_list = []
for i, each_body in enumerate(data.body):
    if len(sent_tokenize(each_body))< 3:
        short_list.append(i)

In [34]:
len(short_list)

29681

In [35]:
data = data.drop(index=short_list).reset_index(drop=True)

In [36]:
data.shape

(526483, 12)

In [37]:
consistent_data, inconsistent_data = train_test_split(data, test_size=0.7, random_state=42, stratify=data.category)

In [38]:
consistent_data.shape, inconsistent_data.shape

((157944, 12), (368539, 12))

In [ ]:
SUBSTITUTE_RATIO = 0.3
substituted_inconsistent_data = pd.DataFrame(columns=['original_id', 'substituted_id', 'title', 'body'])
while not inconsistent_data.empty:
    counter = 0
    target = inconsistent_data.sample() # source
    candidate = inconsistent_data[inconsistent_data.iloc[:, 7:10].isin(target.iloc[0, 7:10].values).values].sample() # substituted resource
    if target.send_date.values[0]==candidate.send_date.values[0]:
        counter += 1
        continue
    target_sentences = sent_tokenize(target.body.values[0])
    candidate_sentences = sent_tokenize(candidate.body.values[0])
    leng_target_sentences = len(target_sentences)
    leng_candidate_sentences = len(candidate_sentences)

    if leng_target_sentences >= leng_candidate_sentences:
        substituted_num = int(np.round(leng_target_sentences*SHUFFLE_RATIO))
    else:
        target, candidate = candidate, target
        target_sentences, candidate_sentences = candidate_sentences, target_sentences
        leng_target_sentences, leng_candidate_sentences = len(target_sentences), len(candidate_sentences)
        substituted_num = int(np.round(len(target_sentences)*SHUFFLE_RATIO))
    #shuffle
    for sel_ind in sorted(random.sample(range(leng_target_sentences), substituted_num)):
        try:
            target_sentences[sel_ind] = candidate_sentences[int(np.ceil(sel_ind * leng_candidate_sentences / leng_target_sentences))]
        except:
            counter += 1
            continue
    
    substituted_inconsistent_data = substituted_inconsistent_data.append({'original_id': target.id.values[0], 'substituted_id': candidate.id.values[0], 'title': target.title.values[0], 'body': target_sentences}, ignore_index=True)
    
    inconsistent_data = inconsistent_data.drop(index=[target.index[0], candidate.index[0]])
    if len(inconsistent_data)%5000 < 2:
        print(f'진행 개수(processed) : {len(substituted_inconsistent_data)}, 남은 개수(remained) : {len(inconsistent_data)}')
    if len(inconsistent_data) < 100:
        print('남은 개수(remained) : ', len(inconsistent_data))
    if counter > 1000:
        break

진행 개수 : 1769, 남은 개수 : 365001
진행 개수 : 4269, 남은 개수 : 360001
진행 개수 : 6769, 남은 개수 : 355001
진행 개수 : 9269, 남은 개수 : 350001
진행 개수 : 11769, 남은 개수 : 345001
진행 개수 : 14269, 남은 개수 : 340001
진행 개수 : 16769, 남은 개수 : 335001
진행 개수 : 19269, 남은 개수 : 330001
진행 개수 : 21769, 남은 개수 : 325001
진행 개수 : 24269, 남은 개수 : 320001
